# Линейные фильтры и свёртки

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
from scipy import ndimage
import os

In [ ]:
# Загружаем тестовое изображение (используем из предыдущей пары)
img_path = 'sample.jpg'

if os.path.exists(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(f"✅ Изображение загружено: {img_path}")
else:
    print("📝 Создаём тестовое изображение...")
    # Создаём более интересное тестовое изображение для демонстрации фильтров
    img = np.zeros((400, 500, 3), dtype=np.uint8)

    # Создаём различные геометрические фигуры для тестирования фильтров
    # Прямоугольники разных цvetов
    img[50:150, 50:200] = [100, 150, 255]  # Светло-синий
    img[200:300, 50:200] = [50, 255, 50]   # Зелёный
    img[50:150, 250:400] = [255, 100, 100] # Красный
    img[200:300, 250:400] = [200, 200, 50] # Жёлто-зелёный

    # Добавляем шум для демонстрации фильтрации
    noise = np.random.randint(0, 50, img.shape, dtype=np.uint8)
    img = cv2.add(img, noise)

    # Добавляем линии для демонстрации детекции границ
    img[175:185, :] = [255, 255, 255]  # Горизонтальная белая линия
    img[:, 225:235] = [255, 255, 255]  # Вертикальная белая линия

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print("✅ Синтетическое изображение создано")

# Преобразуем в grayscale для работы с фильтрами
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Исходное изображение (RGB)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_gray, cmap='gray')
plt.title('Grayscale версия')
plt.axis('off')
plt.tight_layout()
plt.show()



In [ ]:
# ===============================
# ЗАДАНИЕ 1: Реализация свёртки "с нуля"
# ===============================

def manual_convolution_2d(image, kernel, padding='reflect'):
    """
    Реализация 2D свёртки без использования готовых функций

    Args:
        image: входное изображение (2D массив)
        kernel: ядро свёртки (2D массив)
        padding: тип заполнения границ ('reflect', 'constant', 'wrap')

    Returns:
        result: результат свёртки
    """

    # Размеры изображения и ядра
    img_h, img_w = image.shape
    kernel_h, kernel_w = kernel.shape

    # Вычисляем размеры отступов
    pad_h = kernel_h // 2
    pad_w = kernel_w // 2

    # Создаём изображение с отступами
    if padding == 'reflect':
        padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
    elif padding == 'constant':
        padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    elif padding == 'wrap':
        padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='wrap')

    # Инициализируем результат
    result = np.zeros_like(image, dtype=np.float64)

    # Применяем свёртку
    print("⏰ Выполняем свёртку... (может занять время)")

    for y in range(img_h):
        if y % 50 == 0:  # Показываем прогресс
            print(f"   Обработано строк: {y}/{img_h}")

        for x in range(img_w):
            # Извлекаем область изображения размером с ядро
            region = padded[y:y+kernel_h, x:x+kernel_w]

            # Вычисляем взвешенную сумму
            result[y, x] = np.sum(region * kernel)

    # Приводим к правильному диапазону значений
    result = np.clip(result, 0, 255)
    return result.astype(np.uint8)

# Создаём простое ядро размытия (усредняющий фильтр)
blur_kernel = np.ones((5, 5), dtype=np.float32) / 25
print(f"\n🔧 Создано ядро размытия 5x5:")
print(f"Сумма коэффициентов: {np.sum(blur_kernel):.3f}")
print("Ядро:")
print(blur_kernel)

# Применяем нашу реализацию
print("\n⚡ Тестируем нашу реализацию свёртки...")
start_time = time.time()
result_manual = manual_convolution_2d(img_gray, blur_kernel)
manual_time = time.time() - start_time

# Сравниваем с OpenCV
start_time = time.time()
result_opencv = cv2.filter2D(img_gray, -1, blur_kernel)
opencv_time = time.time() - start_time

print(f"⏱️  Время нашей реализации: {manual_time:.3f} сек")
print(f"⏱️  Время OpenCV: {opencv_time:.3f} сек")
print(f"🚀 OpenCV быстрее в {manual_time/opencv_time:.1f} раз")

# Проверяем точность
difference = np.abs(result_manual.astype(float) - result_opencv.astype(float))
print(f"📊 Максимальная разность: {difference.max()}")
print(f"📊 Средняя разность: {difference.mean():.2f}")

# Визуализируем результаты
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

axes[1].imshow(result_manual, cmap='gray')
axes[1].set_title(f'Наша реализация\n({manual_time:.2f} сек)')
axes[1].axis('off')

axes[2].imshow(result_opencv, cmap='gray')
axes[2].set_title(f'OpenCV filter2D\n({opencv_time:.3f} сек)')
axes[2].axis('off')

axes[3].imshow(difference, cmap='hot')
axes[3].set_title(f'Разность\n(макс: {difference.max():.1f})')
axes[3].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Различные типы ядер и их эффекты
# ===============================

# Создаём набор различных ядер
kernels = {
    'Размытие 3x3': np.ones((3, 3)) / 9,

    'Размытие 7x7': np.ones((7, 7)) / 49,

    'Резкость': np.array([[ 0, -1,  0],
                          [-1,  5, -1],
                          [ 0, -1,  0]]),

    'Границы (Собель X)': np.array([[-1, 0, 1],
                                    [-2, 0, 2],
                                    [-1, 0, 1]]),

    'Границы (Собель Y)': np.array([[-1, -2, -1],
                                    [ 0,  0,  0],
                                    [ 1,  2,  1]]),

    'Лапласиан': np.array([[ 0, -1,  0],
                           [-1,  4, -1],
                           [ 0, -1,  0]]),

    'Тиснение': np.array([[-2, -1,  0],
                          [-1,  1,  1],
                          [ 0,  1,  2]]),

    'Контур': np.array([[-1, -1, -1],
                        [-1,  8, -1],
                        [-1, -1, -1]])
}

# Применяем все фильтры
results = {}
for name, kernel in kernels.items():
    print(f"🔄 Применяем фильтр: {name}")

    # Применяем фильтр
    filtered = cv2.filter2D(img_gray, -1, kernel)

    # Для некоторых фильтров нужна нормализация
    if name in ['Границы (Собель X)', 'Границы (Собель Y)', 'Лапласиан', 'Тиснение']:
        # Приводим к диапазону 0-255
        filtered = cv2.convertScaleAbs(filtered)

    results[name] = filtered

# Визуализируем результаты
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

# Исходное изображение
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

# Результаты фильтрации
for i, (name, result) in enumerate(results.items(), 1):
    if i < len(axes):
        axes[i].imshow(result, cmap='gray')
        axes[i].set_title(name)
        axes[i].axis('off')

plt.tight_layout()
plt.show()

# Показываем сами ядра
print("\n🔍 Визуализация ядер фильтров:")

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, (name, kernel) in enumerate(kernels.items()):
    if i < len(axes):
        im = axes[i].imshow(kernel, cmap='RdBu', interpolation='nearest')
        axes[i].set_title(f'{name}\n{kernel.shape}')
        plt.colorbar(im, ax=axes[i], shrink=0.6)

        # Добавляем значения в ячейки для маленьких ядер
        if kernel.shape[0] <= 3:
            for y in range(kernel.shape[0]):
                for x in range(kernel.shape[1]):
                    axes[i].text(x, y, f'{kernel[y,x]:.1f}',
                               ha='center', va='center', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Гауссово размытие и его параметры
# ===============================

def create_gaussian_kernel(size, sigma):
    """Создание гауссова ядра"""
    kernel = np.zeros((size, size))
    center = size // 2

    for i in range(size):
        for j in range(size):
            x, y = i - center, j - center
            kernel[i, j] = np.exp(-(x*x + y*y) / (2 * sigma*sigma))

    # Нормализуем (сумма должна быть 1)
    kernel = kernel / kernel.sum()
    return kernel

# Исследуем влияние параметров
sigmas = [0.5, 1.0, 2.0, 4.0]
kernel_sizes = [3, 7, 11, 15]

print("🧮 Создаём гауссовы ядра с разными параметрами...")

fig, axes = plt.subplots(len(sigmas), len(kernel_sizes) + 1, figsize=(18, 16))

for i, sigma in enumerate(sigmas):
    # Исходное изображение в первом столбце
    axes[i, 0].imshow(img_gray, cmap='gray')
    axes[i, 0].set_title(f'Исходное\n(σ={sigma})')
    axes[i, 0].axis('off')

    for j, ksize in enumerate(kernel_sizes):
        # Создаём гауссово ядро
        gaussian_kernel = create_gaussian_kernel(ksize, sigma)

        # Применяем фильтр
        blurred = cv2.filter2D(img_gray, -1, gaussian_kernel)

        # Сравниваем с встроенной функцией OpenCV
        blurred_opencv = cv2.GaussianBlur(img_gray, (ksize, ksize), sigma)

        # Показываем результат
        axes[i, j+1].imshow(blurred, cmap='gray')

        # Проверяем точность нашей реализации
        diff = np.abs(blurred.astype(float) - blurred_opencv.astype(float))
        max_diff = diff.max()

        axes[i, j+1].set_title(f'Размер={ksize}x{ksize}\nРазность: {max_diff:.1f}')
        axes[i, j+1].axis('off')

plt.suptitle('Гауссово размытие: влияние σ (по строкам) и размера ядра (по столбцам)', fontsize=14)
plt.tight_layout()
plt.show()

# Визуализируем сами гауссовы ядра
print("\n📊 Визуализация гауссовых ядер:")

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

combinations = [(1, 5), (2, 5), (1, 9), (2, 9), (1, 13), (4, 13), (2, 15), (4, 15)]

for i, (sigma, ksize) in enumerate(combinations):
    if i < len(axes):
        kernel = create_gaussian_kernel(ksize, sigma)

        im = axes[i].imshow(kernel, cmap='hot', interpolation='bilinear')
        axes[i].set_title(f'σ={sigma}, размер={ksize}x{ksize}')
        plt.colorbar(im, ax=axes[i], shrink=0.6)

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 4: Сепарабельная свёртка (оптимизация)
# ===============================

def separable_gaussian_blur(image, sigma, ksize):
    """
    Реализация сепарабельного гауссова размытия
    Вместо одной свёртки NxN делаем две свёртки 1xN и Nx1
    """

    # Создаём 1D гауссово ядро
    center = ksize // 2
    kernel_1d = np.zeros(ksize)

    for i in range(ksize):
        x = i - center
        kernel_1d[i] = np.exp(-(x*x) / (2 * sigma*sigma))

    # Нормализуем
    kernel_1d = kernel_1d / kernel_1d.sum()

    # Применяем горизонтальную свёртку
    horizontal_kernel = kernel_1d.reshape(1, -1)
    temp = cv2.filter2D(image, -1, horizontal_kernel)

    # Применяем вертикальную свёртку
    vertical_kernel = kernel_1d.reshape(-1, 1)
    result = cv2.filter2D(temp, -1, vertical_kernel)

    return result, kernel_1d

# Сравнение производительности
test_image = img_gray
sigma = 2.0
ksize = 15

print(f"🏁 Сравнение производительности для σ={sigma}, размер ядра={ksize}x{ksize}")

# Обычная 2D свёртка
gaussian_2d = create_gaussian_kernel(ksize, sigma)
start_time = time.time()
result_2d = cv2.filter2D(test_image, -1, gaussian_2d)
time_2d = time.time() - start_time

# Сепарабельная свёртка
start_time = time.time()
result_separable, kernel_1d = separable_gaussian_blur(test_image, sigma, ksize)
time_separable = time.time() - start_time

# OpenCV GaussianBlur (внутри использует сепарабельную реализацию)
start_time = time.time()
result_opencv = cv2.GaussianBlur(test_image, (ksize, ksize), sigma)
time_opencv = time.time() - start_time

print(f"⏱️  2D свёртка: {time_2d:.4f} сек")
print(f"⏱️  Сепарабельная: {time_separable:.4f} сек")
print(f"⏱️  OpenCV: {time_opencv:.4f} сек")
print(f"🚀 Ускорение сепарабельной: {time_2d/time_separable:.1f} раз")

# Проверяем точность
diff_2d_sep = np.abs(result_2d.astype(float) - result_separable.astype(float))
diff_2d_cv = np.abs(result_2d.astype(float) - result_opencv.astype(float))
diff_sep_cv = np.abs(result_separable.astype(float) - result_opencv.astype(float))

print(f"📊 Разность 2D vs Сепарабельная: макс={diff_2d_sep.max():.2f}, средн={diff_2d_sep.mean():.2f}")
print(f"📊 Разность 2D vs OpenCV: макс={diff_2d_cv.max():.2f}, средн={diff_2d_cv.mean():.2f}")
print(f"📊 Разность Сепарабельная vs OpenCV: макс={diff_sep_cv.max():.2f}, средн={diff_sep_cv.mean():.2f}")

# Визуализируем результаты
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Верхняя строка - результаты
axes[0, 0].imshow(result_2d, cmap='gray')
axes[0, 0].set_title(f'2D свёртка\n{time_2d:.4f} сек')
axes[0, 0].axis('off')

axes[0, 1].imshow(result_separable, cmap='gray')
axes[0, 1].set_title(f'Сепарабельная\n{time_separable:.4f} сек')
axes[0, 1].axis('off')

axes[0, 2].imshow(result_opencv, cmap='gray')
axes[0, 2].set_title(f'OpenCV\n{time_opencv:.4f} сек')
axes[0, 2].axis('off')

# Нижняя строка - ядра и разности
axes[1, 0].imshow(gaussian_2d, cmap='hot')
axes[1, 0].set_title(f'2D ядро {ksize}x{ksize}')

axes[1, 1].plot(kernel_1d)
axes[1, 1].set_title(f'1D ядро (размер {ksize})')
axes[1, 1].grid(True)

axes[1, 2].imshow(diff_2d_sep, cmap='hot')
axes[1, 2].set_title(f'Разность 2D vs Сепарабельная\nмакс: {diff_2d_sep.max():.2f}')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Интерактивное исследование фильтров
# ===============================

# Функция для интерактивного изменения параметров
def interactive_filter_demo(filter_type='gaussian', param1=1.0, param2=5):
    """Интерактивная демонстрация различных фильтров"""

    if filter_type == 'gaussian':
        if param2 % 2 == 0:
            param2 += 1  # Размер ядра должен быть нечётным
        result = cv2.GaussianBlur(img_gray, (param2, param2), param1)
        title = f'Гауссово размытие: σ={param1:.1f}, размер={param2}x{param2}'

    elif filter_type == 'box':
        if param2 % 2 == 0:
            param2 += 1
        kernel = np.ones((param2, param2)) / (param2 * param2)
        result = cv2.filter2D(img_gray, -1, kernel)
        title = f'Box фильтр: размер={param2}x{param2}'

    elif filter_type == 'sharpen':
        strength = param1
        kernel = np.array([[ 0, -1,  0],
                          [-1,  4+strength, -1],
                          [ 0, -1,  0]])
        result = cv2.filter2D(img_gray, -1, kernel)
        result = np.clip(result, 0, 255).astype(np.uint8)
        title = f'Резкость: сила={strength:.1f}'

    elif filter_type == 'edge':
        # Детекция границ с порогом
        edges = cv2.Canny(img_gray, param1, param2)
        result = edges
        title = f'Детекция границ: мин={param1:.0f}, макс={param2:.0f}'

    else:
        result = img_gray
        title = 'Исходное изображение'

    # Отображаем результат
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    axes[0].imshow(img_gray, cmap='gray')
    axes[0].set_title('Исходное изображение')
    axes[0].axis('off')

    axes[1].imshow(result, cmap='gray')
    axes[1].set_title(title)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    return result

# Примеры различных настроек (замените на интерактивные виджеты при необходимости)
print("🎛️  Демонстрация различных настроек фильтров:")

# Гауссово размытие
print("\n1. Гауссово размытие с разными параметрами:")
for sigma in [0.5, 2.0, 4.0]:
    for ksize in [5, 11, 21]:
        result = interactive_filter_demo('gaussian', sigma, ksize)

# Резкость
print("\n2. Фильтры резкости:")
for strength in [1, 3, 5]:
    result = interactive_filter_demo('sharpen', strength, 0)

# Детекция границ
print("\n3. Детекция границ Canny:")
for low, high in [(50, 150), (100, 200), (150, 250)]:
    result = interactive_filter_demo('edge', low, high)

In [ ]:
# ===============================
# ЗАДАНИЕ 6: Создание собственных творческих фильтров
# ===============================

# Создаём набор творческих фильтров
creative_kernels = {
    'Художественное размытие': np.array([[1, 2, 1],
                                         [2, 4, 2],
                                         [1, 2, 1]]) / 16,

    'Тиснение 45°': np.array([[-1, -1,  0],
                             [-1,  0,  1],
                             [ 0,  1,  1]]),

    'Вертикальные полосы': np.array([[-1, 2, -1],
                                    [-1, 2, -1],
                                    [-1, 2, -1]]),

    'Диагональная резкость': np.array([[ 0,  0, -1],
                                      [ 0,  1,  0],
                                      [-1,  0,  0]]),

    'Размытие движения': np.array([[1, 0, 0, 0, 0],
                                  [0, 1, 0, 0, 0],
                                  [0, 0, 1, 0, 0],
                                  [0, 0, 0, 1, 0],
                                  [0, 0, 0, 0, 1]]) / 5
}

# Применяем творческие фильтры
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Исходное изображение
axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

# Применяем каждый фильтр
for i, (name, kernel) in enumerate(creative_kernels.items(), 1):
    if i < len(axes):
        result = cv2.filter2D(img_gray, -1, kernel)

        # Для некоторых фильтров нужна специальная обработка
        if 'Тиснение' in name or 'полосы' in name or 'резкость' in name:
            result = cv2.convertScaleAbs(result)

        axes[i].imshow(result, cmap='gray')
        axes[i].set_title(name)
        axes[i].axis('off')

plt.tight_layout()
plt.show()

## 📝 ДОМАШНЕЕ ЗАДАНИЕ:

1. Создайте свой собственный фильтр (ядро 3x3 или 5x5)
2. Реализуйте функцию для создания фильтра 'Motion Blur' под произвольным углом
3. Сравните время выполнения разных методов размытия на изображении 1000x1000 пикселей
4. Попробуйте создать фильтр, который подчёркивает определённые особенности изображения
